<a href="https://colab.research.google.com/github/edwardoughton/satellite-image-analysis/blob/main/04_01_ggs416_26_02_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛰️ GGS416 Satellite Image Analysis Week 4 🛰️

This week we will cover:

 * Classical image segmentation approaches


# Learning objectives

By the end of this week, students will be able to:

* Explain practical foundations of classical image segmentation methods.
* Implement histogram-based segmentation using Otsu and Multi-Otsu thresholding.
* Apply clustering-based segmentation using K-means in RGB feature space.
* Generate superpixels using SLIC and interpret compactness effects.
* Perform marker-controlled watershed segmentation.
* Apply graph-based segmentation using the Felzenszwalb algorithm.
* Compare segmentation outputs across heterogeneous landscapes (urban, agricultural, forest, coastal).
* Analyze the influence of algorithm parameters on segmentation results.
* Critically evaluate strengths and weaknesses of classical segmentation approaches for satellite imagery.



# Microsoft Copilot Pro

Please sign up:

https://docs.github.com/en/copilot/how-tos/manage-your-account/get-free-access-to-copilot-pro

# Visual Studio Code

Please install before next week:

https://code.visualstudio.com/

# AI advice

* Make sure you are using the newest models, where possible.
* Free apps will have more outdated models.
* Do use your educational email for free access to certain products.

In [ ]:
# Example: Setup and location presets
!pip -q install pystac-client planetary-computer odc-stac rasterio requests

import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.transform import from_origin
from rasterio.crs import CRS

import pystac_client
import planetary_computer
import odc.stac
from pystac.extensions.eo import EOExtension as eo

In [ ]:
# Example: Location presets and search STAC
STAC_API_URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
COLLECTION = "landsat-c2-l2"

# 5 locations (min_lon, min_lat, max_lon, max_lat) in EPSG:4326
LOCATION_PRESETS = {
    "ames_ia": [-93.77, 41.88, -93.47, 42.18],
    "los_angeles_ca": [-118.375, 33.90, -118.075, 34.20],
    "new_orleans_la": [-90.25, 29.80, -89.95, 30.10],
    "phoenix_az": [-112.25, 33.30, -111.95, 33.60],
    "mt_jefferson_or": [-121.95, 44.50, -121.65, 44.80],
}

time_of_interest = "2024-05-01/2024-09-30"
cloud_limit = 10  # percent

DATA_DIR = Path("data_landsat")
DATA_DIR.mkdir(exist_ok=True)

catalog = pystac_client.Client.open(
    STAC_API_URL,
    modifier=planetary_computer.sign_inplace
)

print(catalog)

In [ ]:
# Example: Function to save RGB images as geotifs in a certain folder
def save_rgb_geotiff(data, out_path):
    """
    Save RGB (red/green/blue) from an odc-stac xarray dataset to a GeoTIFF.
    This keeps georeferencing so rasterio cropping works.
    """
    # Build (bands, rows, cols) array for rasterio
    rgb = data[["red", "green", "blue"]].to_array().values  # shape (3, y, x)

    # Get georeferencing from odc-stac dataset
    transform = data.odc.geobox.transform
    crs = data.odc.geobox.crs

    profile = {
        "driver": "GTiff",
        "height": rgb.shape[1],
        "width": rgb.shape[2],
        "count": 3,
        "dtype": rgb.dtype,
        "crs": crs,
        "transform": transform,
        "tiled": True,
        "blockxsize": 256,
        "blockysize": 256,
    }

    # Now write our image to the path we specify
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(rgb)

    return out_path


In [ ]:
# Example: Downloading and cropping multiple images at once
downloaded_rgb_tifs = []

for location_name, bbox in LOCATION_PRESETS.items():

    search = catalog.search(
        collections=[COLLECTION],
        bbox=bbox,
        datetime=time_of_interest,
        query={"eo:cloud_cover": {"lt": cloud_limit}},
    )
    items = search.item_collection()

    # Lecture 2: choose lowest cloud cover
    selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)

    # Lecture 2: load bands with odc-stac
    bands = ["red", "green", "blue"]
    data = odc.stac.stac_load([selected_item], bands=bands, bbox=bbox).isel(time=0)

    out_tif = DATA_DIR / f"{location_name}_rgb.tif"
    save_rgb_geotiff(data, out_tif)

    downloaded_rgb_tifs.append(out_tif)
    print("Saved:", out_tif)


In [ ]:
# Example: Crop images (covered in Lecture 3)
from rasterio.windows import from_bounds
from rasterio.warp import transform_bounds

CROPPED_DIR = DATA_DIR / "cropped" # Set our directory path
CROPPED_DIR.mkdir(exist_ok=True)   # Create our directory

def crop_geotiff_to_bbox(in_path, out_path, bbox_wgs84):
    """
    Crop a GeoTIFF to a lon/lat bbox using the Lecture 3 approach.
    """
    with rasterio.open(in_path) as src:
        # 1) Convert bbox from EPSG:4326 to the raster CRS (UTM for Landsat)
        minlon, minlat, maxlon, maxlat = bbox_wgs84
        minx, miny, maxx, maxy = transform_bounds(
            "EPSG:4326", src.crs, minlon, minlat, maxlon, maxlat, densify_pts=21
        )

        # 2) Lecture 3: build a raster window from bounds
        win = from_bounds(minx, miny, maxx, maxy, src.transform)

        # 3) Read just that window
        data = src.read(window=win)

        # 4) Update metadata and write output (Lecture 3)
        prof = src.profile
        prof.update(
            height=data.shape[1],
            width=data.shape[2],
            transform=rasterio.windows.transform(win, src.transform),
        )

        with rasterio.open(out_path, "w", **prof) as dst:
            dst.write(data)

    return out_path

# Now execute our cropping function, looping over our images
cropped_tifs = []

for tif_path in downloaded_rgb_tifs: # This is the path to our image

    location_name = tif_path.stem.replace("_rgb", "")
    bbox = LOCATION_PRESETS[location_name]

    out_path = CROPPED_DIR / f"{location_name}_rgb_cropped.tif"
    crop_geotiff_to_bbox(tif_path, out_path, bbox)

    cropped_tifs.append(out_path)
    print("Cropped:", out_path.name)


In [ ]:
# Example: Plot images (from Lecture 1)
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, ax in enumerate(axes):
    if i < len(cropped_tifs):
        path = cropped_tifs[i]
        label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

        with rasterio.open(path) as src:
            rgb = src.read([1, 2, 3]).astype("float32")
            rgb = rgb / (rgb.max() + 1e-6)          # normalize for display
            rgb = rgb.transpose(1, 2, 0)            # (rows, cols, bands)

        ax.imshow(rgb)
        ax.set_title(label, fontsize=14)
        ax.axis("off")

        # Panel letter (A, B, C...)
        ax.text(
            0.02, 0.98, chr(ord("A") + i),
            transform=ax.transAxes,
            va="top", ha="left",
            fontsize=14,
            bbox=dict(facecolor="white", alpha=0.7, edgecolor="none", pad=3)
        )
    else:
        ax.axis("off")

fig.suptitle("Landsat RGB — Download + Crop", fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()


Before we get into the segmentation techniques, let us first define a repeatable function for plotting two images side-by-side for easy comparison.

In [ ]:
# Example: Create simple matplotlib function for side-by-side pair comparison
def show_pair(original, segmented, title, cmap=None):
    fig, axes = plt.subplots(1, 2, figsize=(11, 4))
    axes[0].imshow(original)
    axes[0].set_title("Original")
    axes[0].axis("off")

    if segmented.ndim == 2:
        axes[1].imshow(segmented, cmap=cmap or "gray")
    else:
        axes[1].imshow(segmented)
    axes[1].set_title(title)
    axes[1].axis("off")
    plt.tight_layout()
    plt.show()

# Otsu thresholding

Segmentation approach:
- Splits a grayscale image into foreground and background regions.
- Automatically selects a threshold based on the image intensity histogram.

Otsu selects the threshold that splits the image into two substantial groups, maximizing the distance between their average intensities.  

Hence, we find the threshold that best separates foreground from background based on histogram structure.


Key parameters:
- No manual threshold required.
- Input quality depends on histogram separation.

Key papers:
- Otsu, N. (1979). *A Threshold Selection Method from Gray-Level Histograms*. IEEE Trans. SMC, 9(1), 62-66.
- Sezgin, M., & Sankur, B. (2004). *Survey over image thresholding techniques and quantitative performance evaluation*. J. Electronic Imaging, 13(1), 146-165.

In [ ]:
# Example: Apply Otsu threshold and show side-by-side comparison
import numpy as np
import rasterio
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

# Select an image as initial example
path = "data_landsat/cropped/ames_ia_rgb_cropped.tif"

# Load RGB image (.tif)
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype("float32").transpose(1, 2, 0)

# Normalize for display (max values should be ~1)
rgb = rgb / (np.nanmax(rgb) + 1e-6) # 1e-6 is to avoid zero division

# Apply Otsu
gray = rgb2gray(rgb) # RGB to single band grayscale image
t = threshold_otsu(gray) # maximizing the variance between two classes
mask = gray > t # 1 = foreground, 0 = background

otsu_vis = rgb.copy()
otsu_vis[~mask] *= 0.25  # dim only background pixels

show_pair(rgb, otsu_vis, f"Otsu Threshold (t={t:.3f})")

In [ ]:
# Example: Inspect Otsu pixel histogram threshold
import numpy as np
import rasterio
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

path = cropped_tifs[0]

with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype("float32").transpose(1, 2, 0)

rgb = rgb / (np.nanmax(rgb) + 1e-6)
gray = rgb2gray(rgb)

t = threshold_otsu(gray)
mask = gray > t

otsu_vis = rgb.copy()
otsu_vis[~mask] *= 0.25

g = gray[np.isfinite(gray)].ravel()
thr = np.atleast_1d(t).astype(float)

plt.figure(figsize=(8, 4))
plt.hist(g, bins=256, density=True)
for t in thr:
    plt.axvline(t, linewidth=2, color='red')
plt.title(f"Otsu histogram (t={t:.3f} - indicated in red)")
plt.xlabel("Pixel intensity (grayscale)")
plt.ylabel("Density")
plt.tight_layout()
plt.show()


In [ ]:
# Example: Now apply Otsu threshold for all five images
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

image_files = cropped_tifs[:5]  # your cropped GeoTIFFs

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(6, 12))
plt.subplots_adjust(hspace=0.25, wspace=0.05)

for row, path in enumerate(image_files):

    # --- Load GeoTIFF properly ---
    with rasterio.open(path) as src:
        rgb = src.read([1, 2, 3]).astype("float32")  # (3, H, W)
        rgb = rgb.transpose(1, 2, 0)                 # (H, W, 3)

    # Normalize for display
    rgb = rgb / (np.nanmax(rgb) + 1e-6)

    # --- Otsu threshold ---
    gray = rgb2gray(rgb)
    t = threshold_otsu(gray)
    mask = gray > t

    # Dim background for visualization
    otsu_vis = rgb.copy()
    otsu_vis[~mask] *= 0.25

    # Label
    label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

    # --- Plot original ---
    axL = axes[row, 0]
    axL.imshow(rgb)
    axL.set_title(f"{label}", fontsize=12)
    axL.axis("off")

    # --- Plot Otsu result ---
    axR = axes[row, 1]
    axR.imshow(otsu_vis)
    axR.set_title(f"Otsu Threshold", fontsize=12)
    axR.axis("off")

fig.suptitle("Landsat RGB (Left) vs Otsu Segmentation (Right)", fontsize=16)
plt.show()


In [ ]:
# Example: Writing segments to a geopackage
import os
import numpy as np
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu

path_in = "data_landsat/cropped/ames_ia_rgb_cropped.tif"
folder_out = "data_landsat/shapes"
os.makedirs(folder_out, exist_ok=True)   # Create our directory
path_out = os.path.join(folder_out, "agriculture_iowa_ames_shapes_cropped.gpkg")

with rasterio.open(path_in) as src:

    rgb = src.read([1, 2, 3]).astype("float32").transpose(1, 2, 0)

    # Normalize maximum values to ~1.
    rgb = rgb / (np.nanmax(rgb) + 1e-6) # 1e-6 prevents division by zero errors

    # Apply Otsu
    gray = rgb2gray(rgb)
    t = threshold_otsu(gray)
    mask = (gray > t).astype(np.uint8)  # 1 = foreground, 0 = background

    transform = src.transform
    crs = src.crs

# Convert mask to polygons via geodataframe (gdf) and export
geoms = []
vals = []
for geom, val in shapes(mask, mask=(mask == 1), transform=transform):
    geoms.append(shape(geom))
    vals.append(int(val))
gdf = gpd.GeoDataFrame({"class": vals}, geometry=geoms, crs=crs)
gdf.to_file(path_out, driver="GPKG") # Export shapes to .gpkg

# Show final side-by-side plot
show_pair(rgb, mask, "Otsu (single) mask layer")

# Multi-Otsu thresholding (3 classes)

Segmentation approach:
- Extends Otsu to multiple classes.
- Useful for coarse land-cover partitioning (dark/mid/bright regions).



Key parameters:
- `classes=3` (can also try 4 or 5).
- Sensitive to histogram shape and contrast.

Key papers:
- Otsu, N. (1979). *A Threshold Selection Method from Gray-Level Histograms*. IEEE Trans. SMC, 9(1), 62-66.
- Liao, P.-S., Chen, T.-S., & Chung, P.-C. (2001). *A fast algorithm for multilevel thresholding*. JISE, 17(5), 713-727.



In [ ]:
# Example: Multi-Otsu thresholding
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from skimage.color import rgb2gray
from skimage.filters import threshold_multiotsu

# Select one image
path = "data_landsat/cropped/los_angeles_ca_rgb_cropped.tif"

# Load RGB image
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype("float32").transpose(1, 2, 0)

# Normalize
rgb = rgb / (np.nanmax(rgb) + 1e-6)

# Apply Multi-Otsu (3 classes)
gray = rgb2gray(rgb)
t1, t2 = threshold_multiotsu(gray, classes=3)

seg = np.digitize(gray, bins=[t1, t2])

# Visualization: overlay the 3-class segmentation on the RGB
vis = (rgb * 0.35).copy() # Start from a  dimmed version of the original
# Colorize each class
vis[seg == 0] = [0.10, 0.20, 0.80]  # blue # class 0 = dark pixels
vis[seg == 1] = [0.20, 0.80, 0.20]  # green # class 1 = mid pixels
vis[seg == 2] = [0.90, 0.20, 0.20]  # red # class 2 = bright pixels

# Plot
show_pair(rgb, vis, f"Multi-Otsu (3 classes)\nt1={t1:.3f}, t2={t2:.3f}")

In [ ]:
from skimage.filters import threshold_multiotsu

t1, t2 = threshold_multiotsu(gray, classes=3)
seg = np.digitize(gray, bins=[t1, t2]).astype(np.uint8)

# simple visualization (optional)
vis = (rgb * 0.35).copy()
vis[seg == 0] = [0.10, 0.20, 0.80]
vis[seg == 1] = [0.20, 0.80, 0.20]
vis[seg == 2] = [0.90, 0.20, 0.20]

# show_pair(rgb, vis, f"Multi-Otsu (t1={t1:.3f}, t2={t2:.3f})")
# show_hist_with_thresholds(gray, [t1, t2], title="Multi-Otsu histogram (3 classes)")

g = gray[np.isfinite(gray)].ravel()
thr = np.atleast_1d([t1, t2]).astype(float)

plt.figure(figsize=(8, 4))
plt.hist(g, bins=256, density=True)
for t in thr:
    plt.axvline(t, linewidth=2, color="red")
plt.title("Multi-Otsu histogram (3 classes, divided by red thresholds)")
plt.xlabel("Pixel intensity (grayscale)")
plt.ylabel("Density")
plt.tight_layout()
plt.show()


In [ ]:
# Example: Now apply multi-Otsu threshold for all five images
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from skimage.color import rgb2gray
from skimage.filters import threshold_multiotsu

image_files = cropped_tifs[:5]  # your cropped GeoTIFFs

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(6, 12))
plt.subplots_adjust(hspace=0.25, wspace=0.05)

for row, path in enumerate(image_files):

    # Load GeoTIFF properly
    with rasterio.open(path) as src:
        rgb = src.read([1, 2, 3]).astype("float32")  # (3, H, W)
        rgb = rgb.transpose(1, 2, 0)                 # (H, W, 3)

    # Normalize for display
    rgb = rgb / (np.nanmax(rgb) + 1e-6)

    # Multi-Otsu threshold (3 classes)
    gray = rgb2gray(rgb)

    # thresholds will have length classes-1 = 2
    t1, t2 = threshold_multiotsu(gray, classes=3)

    # Convert grayscale into class labels: 0,1,2
    seg = np.digitize(gray, bins=[t1, t2])

    # Visualization: overlay the 3-class segmentation on the RGB
    vis = (rgb * 0.35).copy() # Start from a  dimmed version of the original
    # Colorize each class
    vis[seg == 0] = [0.10, 0.20, 0.80]  # blue # class 0 = dark pixels
    vis[seg == 1] = [0.20, 0.80, 0.20]  # green # class 1 = mid pixels
    vis[seg == 2] = [0.90, 0.20, 0.20]  # red # class 2 = bright pixels

    # Label
    label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

    # --- Plot original ---
    axL = axes[row, 0]
    axL.imshow(rgb)
    axL.set_title(f"{label}", fontsize=12)
    axL.axis("off")

    # --- Plot Multi-Otsu result ---
    axR = axes[row, 1]
    axR.imshow(vis)
    axR.set_title(f"{label} — Multi-Otsu (3 classes)", fontsize=12)
    axR.axis("off")

    # Optional: show thresholds in the console
    print(f"{label}: t1={t1:.3f}, t2={t2:.3f}")

fig.suptitle("Landsat RGB (Left) vs Multi-Otsu 3-Class Segmentation (Right)", fontsize=16)
plt.show()


In [ ]:
# Example: Writing segments to a geopackage
import os
import numpy as np
import rasterio
from rasterio.features import shapes
import geopandas as gpd
from shapely.geometry import shape
from skimage.color import rgb2gray
from skimage.filters import threshold_multiotsu

path_in = "data_landsat/cropped/los_angeles_ca_rgb_cropped.tif"
folder_out = "data_landsat/shapes"
os.makedirs(folder_out, exist_ok=True)   # Create our directory
path_out = os.path.join(folder_out, "los_angeles_ca_rgb_cropped.gpkg")

with rasterio.open(path_in) as src:

    rgb = src.read([1, 2, 3]).astype("float32").transpose(1, 2, 0)

    # Normalize maximum values to ~1.
    rgb = rgb / (np.nanmax(rgb) + 1e-6) # 1e-6 prevents division by zero errors

    # Multi-Otsu (3 classes)
    gray = rgb2gray(rgb)
    t1, t2 = threshold_multiotsu(gray, classes=3)

    # Class labels: 0, 1, 2
    seg = np.digitize(gray, bins=[t1, t2]).astype(np.uint8)

    transform = src.transform
    crs = src.crs

# Convert segmentation to polygons (keeps class value per polygon)
geoms = []
classes = []
for geom, val in shapes(seg, transform=transform):
    geoms.append(shape(geom))
    classes.append(int(val))
gdf = gpd.GeoDataFrame({"class": classes}, geometry=geoms, crs=crs)
gdf.to_file(path_out, driver="GPKG") # Export shapes to .gpkg

# Visualization: overlay the 3-class segmentation on the RGB
vis = (rgb * 0.35).copy() # Start from a  dimmed version of the original
# Colorize each class
vis[seg == 0] = [0.10, 0.20, 0.80]  # blue # class 0 = dark pixels
vis[seg == 1] = [0.20, 0.80, 0.20]  # green # class 1 = mid pixels
vis[seg == 2] = [0.90, 0.20, 0.20]  # red # class 2 = bright pixels

# Show final side-by-side plot
show_pair(rgb, vis, "Multi-Otsu mask layer")

In [ ]:
# Example: Viewing close up of New Orleans Multi-Otsu
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from skimage.color import rgb2gray
from skimage.filters import threshold_multiotsu

path = "data_landsat/cropped/new_orleans_la_rgb_cropped.tif"

# Load GeoTIFF
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype("float32")
    rgb = rgb.transpose(1, 2, 0)

# Normalize
rgb = rgb / (np.nanmax(rgb) + 1e-6)

# Multi-Otsu (3 classes)
gray = rgb2gray(rgb)
thresholds = threshold_multiotsu(gray, classes=3)
seg = np.digitize(gray, bins=thresholds)

# Plot full-size
plt.figure(figsize=(10, 10))
plt.imshow(seg, cmap="viridis")
plt.title("New Orleans — Multi-Otsu (3 Classes)", fontsize=16)
plt.axis("off")
plt.show()

print(f"Thresholds: {thresholds}")

# K-means (RGB clustering)

Segmentation approach:
- Clusters pixels in color space.
- Each pixel is assigned to nearest color centroid.

So, K-means segmentation groups pixels based on similarity in color space.

For RGB images:
- Each pixel is treated as a 3D vector (see 3D plot below where each pixel conforms to $
  x_i = [R_i, G_i, B_i]
  $).
- Hence, pixels are clustered in this 3D space.
- Each pixel is assigned to the nearest cluster centroid.

Unlike Otsu (which uses intensity only), K-means operates in multivariate feature space.

Key parameters:
- `k`: number of clusters.
- Iterations and initialization affect stability.


Key papers:
- MacQueen, J. (1967). *Some Methods for classification and Analysis of Multivariate Observations*. Proc. 5th Berkeley Symposium.
- Lloyd, S. (1982). *Least squares quantization in PCM*. IEEE Trans. Information Theory, 28(2), 129-137.

In [ ]:
import numpy as np
import rasterio
from sklearn.cluster import KMeans

# Select one image
path = cropped_tifs[0]

# Load RGB
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

# Percentile stretch to uint8 (pixel value normalization)
p2, p98 = np.nanpercentile(rgb, (2, 98))
rgb01 = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)
rgb8 = (rgb01 * 255).astype(np.uint8)

# K-means using scikit-learn
h, w, _ = rgb8.shape
X = rgb8.reshape(-1, 3)

k = 5
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = kmeans.fit_predict(X)

# Replace each pixel with its centroid color
segmented = kmeans.cluster_centers_[labels].reshape(h, w, 3).astype(np.uint8)

# Show result
show_pair(rgb8, segmented, f"K-means RGB (k={k})")


We are able to explore the number of pixels which fall in each group, which helps to inform us whether:

* The clusters are balanced
* One cluster dominates
* If k is too large (clusters become too small)

We can do that as follows:

In [ ]:
# Example: Exploring cluster size distribution and centroid colors
import numpy as np
import matplotlib.pyplot as plt

counts = np.bincount(labels)
plt.figure(figsize=(6,4))  # Plot k-means cluster sizes
plt.bar(range(k), counts)
plt.xlabel("Cluster ID")
plt.ylabel("Number of Pixels")
plt.title("K-means Cluster Sizes")
plt.show()

centroids = kmeans.cluster_centers_.astype(np.uint8)
plt.figure(figsize=(8,2))  # Plot color of cluster centroids
plt.imshow([centroids])
plt.yticks([])
plt.xticks(range(k))
plt.title("Cluster Centroid Colors")
plt.show()


If it is still not clear how the Otsu methods are one dimensional (1D), and k-means is three dimensional (3D), see the 3D plot below to illustrate the concept.

Pixels are plotted based on the r, g, b values, against axis x, y, z.

In [ ]:
# Example: Plot the pixels in 3D space (for red, green, blue values)
from mpl_toolkits.mplot3d import Axes3D

sample_idx = np.random.choice(X.shape[0], size=5000, replace=False)
X_sample = X[sample_idx]
labels_sample = labels[sample_idx]

fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_sample[:,0], X_sample[:,1], X_sample[:,2],
           c=labels_sample, s=2)
ax.set_xlabel("R")
ax.set_ylabel("G")
ax.set_zlabel("B", rotation=0, labelpad=1)

plt.title("K-means clusters in 3D RGB space")
plt.show()

In [ ]:
# Example: K-means RGB segmentation for all 5 images
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from sklearn.cluster import KMeans

# Apply to all 5 cropped GeoTIFFs and then plot
image_files = cropped_tifs[:5]

# Set up our matplotlib plot
fig, axes = plt.subplots(nrows=len(image_files), ncols=2, figsize=(6, 12))
plt.subplots_adjust(hspace=0.25, wspace=0.05)

k = 5 # Number of desired clusters

for row, path in enumerate(image_files):

    # Read GeoTIFF (bands, H, W) -> (H, W, 3)
    with rasterio.open(path) as src:
        rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

    # Percentile stretch to uint8 for display + clustering
    p2, p98 = np.nanpercentile(rgb, (2, 98))
    rgb01 = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)
    rgb8 = (rgb01 * 255).astype(np.uint8)

    # K-means (scikit-learn)
    h, w, _ = rgb8.shape
    X = rgb8.reshape(-1, 3)
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = km.fit_predict(X)
    kmeans_rgb = km.cluster_centers_[labels].reshape(h, w, 3).astype(np.uint8)
    label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

    # Left: original
    axL = axes[row, 0]
    axL.imshow(rgb8)
    axL.set_title(f"{label}", fontsize=12)
    axL.axis("off")

    # Right: k-means quantized
    axR = axes[row, 1]
    axR.imshow(kmeans_rgb)
    axR.set_title(f"K-means (k={k})", fontsize=12)
    axR.axis("off")

fig.suptitle("Landsat RGB (Left) vs K-means RGB Clustering (Right)", fontsize=16)
plt.show()

# Simple Linear Iterative Clustering (SLIC) superpixels

Simple Linear Iterative Clustering (SLIC) groups pixels using a combined color + spatial distance metric.

By "superpixels" we are referring to regionally aggregated areas, based on the underlying pixel values.

Segmentation approach:
- Groups nearby pixels into compact superpixels.
- Balances color similarity and spatial proximity.

You can think of SLIC as k-means taking place in 5D space, where we develop a 5D feature vector ($\mathbf{f}_i$) as follows:

$$
\mathbf{f}_i = (L_i, a_i, b_i, x_i, y_i)
$$

where:

- $(L_i, a_i, b_i$) are the pixel values in CIELAB color space.
- $(x_i, y_i$) are the spatial coordinates of the pixel.

Note, CIELAB is the color space defined by the International Commission on Illumination (CIE) consisting of $L*a*b*$.

Key parameters:
- `n_segments`: number of superpixels.
- `compactness`: spatial regularity vs boundary adherence.


Key papers:
- Achanta, R., et al. (2012). *SLIC Superpixels Compared to State-of-the-art Superpixel Methods*. IEEE TPAMI, 34(11), 2274-2282.
- Stutz, D., Hermans, A., & Leibe, B. (2018). *Superpixels: An Evaluation of the State-of-the-Art*. CVIU, 166, 1-27.

In [ ]:
import numpy as np
import rasterio
from skimage.segmentation import slic, mark_boundaries

# Select one image
path = "data_landsat/cropped/phoenix_az_rgb_cropped.tif"

# Load RGB ---
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

# Percentile stretch -> [0,1] float (helps SLIC + display)
p2, p98 = np.nanpercentile(rgb, (2, 98))
rgb_float = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

# SLIC superpixels function from sci-kit image
slic_labels = slic(
    rgb_float,
    n_segments=250,
    compactness=12.0,
    sigma=1.0,
    start_label=1
)

# Overlay boundaries on the image
overlay = mark_boundaries(rgb_float, slic_labels, color=(1, 1, 0))

# Show before/after using show_pair()
show_pair(rgb_float, overlay, "SLIC boundaries for Phoenix, AZ")


You can view the superpixel label map to get a better view of regional partitioning processes.

In [ ]:
# Example: Superpixel label map
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.imshow(slic_labels, cmap="tab20")
plt.title("SLIC Label Map (Superpixel IDs)")
plt.axis("off")
plt.show()


Importantly, the SLIC starts from a regular grid before iterative refinement, as illustrated below.

In [ ]:
# Example: Initial SLIC grid
h, w = rgb_float.shape[:2]
S = int(np.sqrt((h*w)/250))  # example approximate grid spacing

plt.imshow(rgb_float)
for y in range(0, h, S):
    plt.axhline(y)
for x in range(0, w, S):
    plt.axvline(x)
plt.title("Example approximate initial grid (before iteration)")
plt.axis("off")
plt.show()


Be aware there are other parameters which affect the partitioning process, including compactness.

View the example below to understand how this affects the image.

In [ ]:
# Example: Comparing different compactness values
labels_low = slic(rgb_float, n_segments=250, compactness=8)
labels_high = slic(rgb_float, n_segments=250, compactness=30)

overlay_low = mark_boundaries(rgb_float, labels_low)
overlay_high = mark_boundaries(rgb_float, labels_high)

show_pair(overlay_low, overlay_high,"Low compactness (left) vs High compactness (right)")


See the feature space vector below (`features`), which when printed shows the five values (for the spatial coordinates and RGB values).

In [ ]:
# Example: Inspect the feature space
h, w = rgb_float.shape[:2]
xx, yy = np.meshgrid(np.arange(w), np.arange(h))

features = np.dstack([rgb_float, xx / w, yy / h])

print(features[1])

We can replace each superpixel with its mean color to simplify the image, and conceptually illustrate SLIC.

In [ ]:
# Example: Color superpixels with the mean color in each
recon = np.zeros_like(rgb_float)

for label in np.unique(slic_labels):
    mask = slic_labels == label
    recon[mask] = rgb_float[mask].mean(axis=0)

show_pair(rgb_float, recon, "Original vs Superpixel Mean Color")


In [ ]:
# Example: Apply SLIC approach to all five images
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from skimage.segmentation import slic, mark_boundaries

image_files = cropped_tifs[:5]

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(6, 12))
plt.subplots_adjust(hspace=0.25, wspace=0.05)

for row, path in enumerate(image_files):

    # Load GeoTIFF
    with rasterio.open(path) as src:
        rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

    # Percentile stretch -> [0,1] float (helps SLIC + display)
    p2, p98 = np.nanpercentile(rgb, (2, 98))
    rgb_float = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

    # SLIC superpixels
    slic_labels = slic(
        rgb_float,
        n_segments=250,
        compactness=12.0,
        sigma=1.0,
        start_label=1
    )

    # Draw boundaries on top of the image (yellow-ish boundary color)
    boundary_overlay = mark_boundaries(rgb_float, slic_labels, color=(1, 1, 0))

    # Label
    label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

    # Plot original
    axL = axes[row, 0]
    axL.imshow(rgb_float)
    axL.set_title(f"{label}", fontsize=12)
    axL.axis("off")

    # Plot superpixels overlay
    axR = axes[row, 1]
    axR.imshow(boundary_overlay)
    axR.set_title(f"SLIC superpixels", fontsize=12)
    axR.axis("off")

fig.suptitle("Landsat RGB (Left) vs SLIC Superpixels (Right)", fontsize=16)
plt.show()


# Watershed segmentation

Watershed segmentation finds region boundaries by simulating how water would fill intensity basins in an image.

This segmentation approach:
- Treats the gradient magnitude in an image as a topographic surface and performs "region flooding" from predefined markers.
- Produces region boundaries from gradient basins.

Think of this as pouring water onto a surface, with the water filling up the low areas first. Where water from different basins meets, you build a dam (e.g., treating the image like a "topographic surface").

Brighter pixels represent higher elevations, and darker pixels represent lower elevations.

Key parameters:
- Marker strategy strongly affects output.
- `min_distance` in peak detection controls seed density.


Key papers:
- Vincent, L., & Soille, P. (1991). *Watersheds in digital spaces: An efficient algorithm based on immersion simulations*. IEEE TPAMI, 13(6), 583-598.
- Roerdink, J. B. T. M., & Meijster, A. (2000). *The Watershed Transform: Definitions, Algorithms and Parallelization Strategies*. Fundamenta Informaticae, 41, 187-228.

In [ ]:
# Example: Watershed analysis

import numpy as np
import rasterio
from scipy import ndimage as ndi
from skimage.color import rgb2gray
from skimage.feature import peak_local_max
from skimage.filters import sobel, threshold_otsu
from skimage.segmentation import watershed, mark_boundaries

# Pick one image
path = "data_landsat/cropped/mt_jefferson_or_rgb_cropped.tif"

# Load + normalize to [0,1]
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)
p2, p98 = np.nanpercentile(rgb, (2, 98))
rgb = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

# Watershed (minimal pipeline)
gray = rgb2gray(rgb)
mask = gray > threshold_otsu(gray)
dist = ndi.distance_transform_edt(mask)
coords = peak_local_max(dist, min_distance=15, labels=mask)
markers = np.zeros_like(gray, dtype=np.int32)
markers[tuple(coords.T)] = np.arange(1, len(coords) + 1) if len(coords) else 1
markers = ndi.label(markers > 0)[0]
labels = watershed(sobel(gray), markers, mask=mask)

# Overlay boundaries + plot
overlay = mark_boundaries(rgb, labels, color=(1, 0, 0))
show_pair(rgb, overlay, "Watershed (boundaries)")


The following panel plot shows the intermediate steps for how this is achieved:

* Grayscale - This is the essentially the height map
* Gradient - Edges become mountains, with flooding avoids crossing these ridges
* Mask - We only segment where we think the object area is
* Distance transform - Centers of objects are deepest basins
* Markers - These are the starting points for flooding
* Overlay - Dams where floods meet boundaries

In [ ]:
# Example: Intermediate steps
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from scipy import ndimage as ndi
from skimage.color import rgb2gray
from skimage.filters import sobel, threshold_otsu
from skimage.feature import peak_local_max
from skimage.segmentation import watershed, mark_boundaries

# pick one image
path = "data_landsat/cropped/mt_jefferson_or_rgb_cropped.tif"

# load + normalize
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)
p2, p98 = np.nanpercentile(rgb, (2, 98))
rgb = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

# 1) terrain
gray = rgb2gray(rgb)

# 2) ridges
grad = sobel(gray)

# 3) mask + distance (basins)
t = threshold_otsu(gray)
mask = gray > t
dist = ndi.distance_transform_edt(mask)

# 4) markers (seeds)
coords = peak_local_max(dist, min_distance=15, labels=mask)
markers = np.zeros_like(gray, dtype=np.int32)
if len(coords):
    markers[tuple(coords.T)] = np.arange(1, len(coords) + 1)
else:
    markers[gray.shape[0]//2, gray.shape[1]//2] = 1
markers = ndi.label(markers > 0)[0]

# 5) watershed result
labels = watershed(grad, markers, mask=mask)
overlay = mark_boundaries(rgb, labels, color=(1, 0, 0))

# Plot
fig, ax = plt.subplots(2, 3, figsize=(14, 8))

ax[0,0].imshow(rgb);        ax[0,0].set_title("Original RGB");        ax[0,0].axis("off")
ax[0,1].imshow(gray, cmap="gray"); ax[0,1].set_title("Grayscale (terrain)"); ax[0,1].axis("off")
ax[0,2].imshow(grad, cmap="gray"); ax[0,2].set_title("Gradient (ridges)");   ax[0,2].axis("off")

ax[1,0].imshow(mask, cmap="gray"); ax[1,0].set_title("Mask (where to segment)"); ax[1,0].axis("off")
ax[1,1].imshow(dist, cmap="gray"); ax[1,1].set_title("Distance transform (basins)"); ax[1,1].axis("off")
ax[1,2].imshow(overlay);           ax[1,2].set_title("Watershed boundaries"); ax[1,2].axis("off")

plt.tight_layout()
plt.show()

print(f"Otsu threshold (for mask): {t:.3f}")
print(f"Number of markers (seeds): {markers.max()}")
print(f"Number of watershed regions: {labels.max()}")


In [ ]:
# Example: Watershed segmentation for all five images
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from scipy import ndimage as ndi
from skimage.color import rgb2gray
from skimage.feature import peak_local_max
from skimage.filters import sobel, threshold_otsu
from skimage.segmentation import watershed, mark_boundaries

image_files = cropped_tifs[:5]

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(6, 12))
plt.subplots_adjust(hspace=0.25, wspace=0.05)

for row, path in enumerate(image_files):

    # --- Load GeoTIFF (RGB) ---
    with rasterio.open(path) as src:
        rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

    # Percentile stretch -> [0,1] float for display/segmentation
    p2, p98 = np.nanpercentile(rgb, (2, 98))
    rgb_float = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

    # --- Watershed pipeline (like your snippet) ---
    gray = rgb2gray(rgb_float)

    gradient = sobel(gray)
    otsu_t = threshold_otsu(gray)

    mask = gray > otsu_t
    distance = ndi.distance_transform_edt(mask)

    # local maxima as markers
    marker_coords = peak_local_max(distance, min_distance=15, labels=mask)

    markers = np.zeros_like(gray, dtype=np.int32)
    if len(marker_coords) > 0:
        markers[tuple(marker_coords.T)] = np.arange(1, len(marker_coords) + 1)
    else:
        h, w = gray.shape
        markers[h // 2, w // 2] = 1

    markers, _ = ndi.label(markers > 0)

    ws_labels = watershed(gradient, markers, mask=mask)

    # boundaries overlay (red)
    overlay = mark_boundaries(rgb_float, ws_labels, color=(1, 0, 0))

    # Label
    label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

    # --- Plot original ---
    axL = axes[row, 0]
    axL.imshow(rgb_float)
    axL.set_title(f"{label}", fontsize=12)
    axL.axis("off")

    # --- Plot watershed overlay ---
    axR = axes[row, 1]
    axR.imshow(overlay)
    axR.set_title(f"Watershed", fontsize=12)
    axR.axis("off")

fig.suptitle("Landsat RGB (Left) vs Watershed Segmentation (Right)", fontsize=16)
plt.show()


# Felzenszwalb graph segmentation

Graph driven region merging via local intensity differences.

This segmentation approach:
- Builds a graph where each pixel is a node
- Connects neighboring pixels with weighted edges
- Sorts edges by weight (difference)
- Merges regions if the boundary evidence is weak


Key parameters:
- `scale`: larger gives larger segments.
- `min_size`: removes tiny noisy segments.



Key papers:
- Felzenszwalb, P. F., & Huttenlocher, D. P. (2004). *Efficient Graph-Based Image Segmentation*. IJCV, 59(2), 167-181.
- Shi, J., & Malik, J. (2000). *Normalized Cuts and Image Segmentation*. IEEE TPAMI, 22(8), 888-905.

In [ ]:
# Example: Felzenszwalb graph-based segmentation
import numpy as np
import rasterio
from skimage.segmentation import felzenszwalb, mark_boundaries

# Pick one image
path = "data_landsat/cropped/new_orleans_la_rgb_cropped.tif"

# Load RGB
with rasterio.open(path) as src:
    rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

# Normalize to [0,1] for segmentation + display
p2, p98 = np.nanpercentile(rgb, (2, 98))
rgb = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

# Felzenszwalb graph-based segmentation
labels = felzenszwalb(rgb, scale=175, sigma=0.8, min_size=80)

# Overlay boundaries
overlay = mark_boundaries(rgb, labels, color=(0, 1, 1))

# Show before/after using your helper
show_pair(rgb, overlay, "Felzenszwalb (boundaries)")


We can visualize the gradient strength between pixels as follows.

Bright pixels show a strong edge, indicating a likely boundary.

Whereas, dark pixels show a weak edge, suggesting a likely merge region.

In [ ]:
# Example: Visualize gradient (edge) strength
from skimage.filters import sobel
import matplotlib.pyplot as plt

grad = sobel(rgb.mean(axis=2))

plt.imshow(grad, cmap="gray")
plt.title("Gradient (edge strength)")
plt.axis("off")
plt.show()


Felzenszwalb applies Gaussian smoothing internally to control noise, as illustrated below.  

In [ ]:
# Example: Applying Gaussian smoothing to reduce pixel noise
from skimage.filters import gaussian

smoothed = gaussian(rgb, sigma=0.8)

show_pair(rgb, smoothed, "Gaussian smoothing (sigma=0.8)")

Next, we can view the pixel map.

In [ ]:
# Example: Felzenszwalb pixel map for New Orleans
plt.imshow(labels, cmap="tab20")
plt.title("Felzenszwalb Label Map")
plt.axis("off")
plt.show()


Next, we can replace each region with its mean color (indicating segments are coherent regions). The algorithm simplifies the image.


In [ ]:
# Example: Felzenszwalb segment mean reconstruction
recon = np.zeros_like(rgb)

for lab in np.unique(labels):
    mask = labels == lab
    recon[mask] = rgb[mask].mean(axis=0)

show_pair(rgb, recon, "Region Mean Reconstruction")


We can manipulate the scale variable to see how aggressively regions merge.

In [ ]:
# Example: Varying the Felzenszwalb scale parameter
labels_small = felzenszwalb(rgb, scale=50, sigma=0.8, min_size=80)
labels_large = felzenszwalb(rgb, scale=300, sigma=0.8, min_size=80)

overlay_small = mark_boundaries(rgb, labels_small)
overlay_large = mark_boundaries(rgb, labels_large)

show_pair(overlay_small, overlay_large,
          "Small scale (left) vs Large scale (right)")


In [ ]:
# Example: Felzenszwalb graph-based segmentation for all five images
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from skimage.segmentation import felzenszwalb, mark_boundaries

image_files = cropped_tifs[:5]

fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(6, 12))
plt.subplots_adjust(hspace=0.25, wspace=0.05)

for row, path in enumerate(image_files):

    # Load GeoTIFF (RGB)
    with rasterio.open(path) as src:
        rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

    # Percentile stretch -> [0,1] float for display/segmentation
    p2, p98 = np.nanpercentile(rgb, (2, 98))
    rgb_float = np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

    # Felzenszwalb segmentation
    felz_labels = felzenszwalb(
        rgb_float,
        scale=175,     # larger => larger segments
        sigma=0.8,     # smoothing
        min_size=80    # minimum segment size
    )

    # Boundary overlay (cyan)
    overlay = mark_boundaries(rgb_float, felz_labels, color=(0, 1, 1))

    # Label
    label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

    # Plot original
    axL = axes[row, 0]
    axL.imshow(rgb_float)
    axL.set_title(f"{label}", fontsize=12)
    axL.axis("off")

    # Plot Felzenszwalb overlay
    axR = axes[row, 1]
    axR.imshow(overlay)
    axR.set_title(f"Felzenszwalb", fontsize=12)
    axR.axis("off")

fig.suptitle("Landsat RGB (Left) vs Felzenszwalb Graph Segmentation (Right)", fontsize=16)
plt.show()


In [ ]:
# Example: Putting all segmentation techniques together for inspection
import numpy as np
import matplotlib.pyplot as plt
import rasterio

from scipy import ndimage as ndi
from sklearn.cluster import KMeans

from skimage.color import rgb2gray
from skimage.filters import threshold_otsu, threshold_multiotsu, sobel
from skimage.feature import peak_local_max
from skimage.segmentation import slic, felzenszwalb, watershed, mark_boundaries

def stretch01(rgb):
    """Percentile stretch to [0,1] float (consistent across methods)."""
    p2, p98 = np.nanpercentile(rgb, (2, 98))
    return np.clip((rgb - p2) / (p98 - p2 + 1e-6), 0, 1)

def kmeans_quantize_rgb(rgb01, k=5, seed=42):
    """
    K-means color quantization using scikit-learn (short + standard).
    Input: rgb01 float in [0,1], shape (H,W,3)
    Output: rgb01 float in [0,1], each pixel replaced by its centroid color
    """
    h, w, _ = rgb01.shape
    X = (rgb01.reshape(-1, 3) * 255).astype(np.float32)  # work in 0..255 space

    km = KMeans(n_clusters=k, random_state=seed, n_init=10)
    labels = km.fit_predict(X)

    out = km.cluster_centers_[labels].reshape(h, w, 3) / 255.0
    return np.clip(out, 0, 1)

ordered_methods = [
    "otsu_thresholding",
    "multi_otsu_3class",
    "kmeans_rgb",
    "slic_superpixels",
    "watershed",
    "felzenszwalb",
]

nrows = len(cropped_tifs)
ncols = 1 + len(ordered_methods)

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4 * nrows))
if nrows == 1:
    axes = axes[None, :]  # keep 2D indexing

for r, path in enumerate(cropped_tifs):

    with rasterio.open(path) as src: # Load RGB
        rgb = src.read([1, 2, 3]).astype(np.float32).transpose(1, 2, 0)

    rgb01 = stretch01(rgb)
    gray = rgb2gray(rgb01)

    location_label = path.stem.replace("_rgb_cropped", "").replace("_", " ").title()

    # Add original
    axes[r, 0].imshow(rgb01)
    axes[r, 0].set_title(f"{location_label}\nOriginal", fontsize=11)
    axes[r, 0].axis("off")

    # 1) Add Otsu (binary -> dim background)
    t = threshold_otsu(gray)
    mask = gray > t
    otsu_vis = rgb01.copy()
    otsu_vis[~mask] *= 0.25

    # 2) Add Multi-Otsu (3 classes -> simple class colors)
    t1, t2 = threshold_multiotsu(gray, classes=3)
    seg3 = np.digitize(gray, bins=[t1, t2]).astype(np.uint8)
    multi_vis = (rgb01 * 0.35).copy()
    multi_vis[seg3 == 0] = [0.10, 0.20, 0.80]  # blue
    multi_vis[seg3 == 1] = [0.20, 0.80, 0.20]  # green
    multi_vis[seg3 == 2] = [0.90, 0.20, 0.20]  # red

    # 3) Add K-means RGB
    kmeans_vis = kmeans_quantize_rgb(rgb01, k=5, seed=42)

    # 4) Add SLIC superpixels (boundaries)
    slic_labels = slic(rgb01, n_segments=250, compactness=12.0, sigma=1.0, start_label=1)
    slic_vis = mark_boundaries(rgb01, slic_labels, color=(1, 1, 0))  # yellow

    # 5) Add Watershed (mask->distance->markers->watershed)
    ws_mask = gray > threshold_otsu(gray)
    dist = ndi.distance_transform_edt(ws_mask)
    coords = peak_local_max(dist, min_distance=15, labels=ws_mask)

    markers = np.zeros_like(gray, dtype=np.int32)
    markers = ndi.label(markers > 0)[0]

    ws_labels = watershed(sobel(gray), markers, mask=ws_mask)
    ws_vis = mark_boundaries(rgb01, ws_labels, color=(1, 0, 0))  # red

    # 6) Add Felzenszwalb (boundaries)
    felz_labels = felzenszwalb(rgb01, scale=175, sigma=0.8, min_size=80)
    felz_vis = mark_boundaries(rgb01, felz_labels, color=(0, 1, 1))  # cyan

    results_row = {
        "otsu_thresholding": otsu_vis,
        "multi_otsu_3class": multi_vis,
        "kmeans_rgb": kmeans_vis,
        "slic_superpixels": slic_vis,
        "watershed": ws_vis,
        "felzenszwalb": felz_vis,
    }

    for c, method in enumerate(ordered_methods, start=1):
        axes[r, c].imshow(results_row[method])
        axes[r, c].set_title(method.replace("_", " "), fontsize=11)
        axes[r, c].axis("off")

fig.suptitle("Landsat: Original + 6 Segmentation Methods (Rows = Locations)", fontsize=18, y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.98])
plt.show()
